In [ ]:
# Compute Stimulus spectrograms

from scipy.io import wavfile
from scipy.signal import spectrogram
stimname = 'I_scaled_burung'
stimfile = '/home/brad/krista_stims/I_40k.wav'

fs, stimdat = wavfile.read(stimfile)
print(fs)

winlen = 10 # ms
winlen_samps = int(winlen*fs/1000.)
#winlen_samps = 256
winlen_samps = 128
nfft = 128
noverlap = int(winlen_samps/2)
nt_win = 5
ovr = np.round(winlen_samps*pcover, )
f, t, Sxx = spectrogram(stimdat, fs, nperseg=winlen_samps, noverlap=ovr)

def downsample_freq(dat):
    return (dat[1::2, :] + dat[:-1:2, :])/2

def downsample_t(dat):
    if len(dat.shape) == 1:
        return (dat[1::2] + dat[:-1:2])/2
    return (dat[:, 1::2] + dat[:, :-1:2])/2

avgSxx = downsample_freq(downsample_freq(downsample_freq(Sxx)))
avgSxx = downsample_t(downsample_t(avgSxx))
t = downsample_t(downsample_t(t))
#t = t[::3]
avgSxx = np.log10(avgSxx)
avgSxx = avgSxx - np.tile(np.mean(avgSxx, axis=1)[:, np.newaxis], (1, avgSxx.shape[1]))
avgSxx = np.divide(avgSxx, np.tile(np.std(avgSxx, axis=1)[:, np.newaxis], (1, avgSxx.shape[1])))
stim = np.zeros((nt_win*avgSxx.shape[0], avgSxx.shape[1]))
for win in range(nt_win, avgSxx.shape[1]):
    stim[:, win] = np.reshape(avgSxx[:, win-nt_win:win], (nt_win*avgSxx.shape[0]))

In [ ]:
# Extract relevant data
from ephys import core, events

bp = bps['B1056']

spikes = core.load_spikes(bp)
fs_spikes = core.load_fs(bp)
trials = events.load_trials(bp)
clusters = core.load_clusters(bp)

# Extract trials from given stimulus
stim_trials = trials[trials['stimulus'] == stimname]

# get segment times for trial of interest
trial = 0

#
trial_bounds = (stim_trials.iloc[trial]['time_samples'], stim_trials.iloc[trial]['stimulus_end'])
clusters_to_use = clusters[clusters['quality'] == 'Good']
clusters_list = clusters_to_use['cluster'].unique()
spikes = spikes[spikes['cluster'].isin(list(clusters_list))]
segment = tp2.get_segment((0, trial_bounds[1]-trial_bounds[0]), fs_spikes, [0,0])
nclus = len(clusters_list)

# convert time to samples:
t_samp = t*fs_spikes

In [ ]:
# get the population tensor
poptens = tp2.build_poptens_given_windows(stim_trials, spikes, t_samp, clusters_list, segment)